In [34]:
!pip install requests owslib html2text openai supabase python-dotenv pandas scikit-learn matplotlib

  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.6/10.6 MB 77.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 75.5 MB/s eta 0:00:00
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 47.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.7/20.7 MB 76.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15/15 [matplotlib]5 [matplotlib]n]


### Save the requirements

In [36]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [25]:
# === Config ===

FILE_NAME_CSW = "./data/csw_records.json"
BASE_DIR_DATASETS = "./data/datasets"
UPDATES_LOG_DIR = "./data/logs/updates"
DELETIONS_LOG_DIR = "./data/logs/deletions"
UPDATE_INTERVAL = 10
BASE_DIR_MARKDOWNS = "./data/markdowns"


In [26]:
from datetime import datetime, timedelta

# === CONFIGS DATE===
# ( you can hardcode a specific date)
TODAY = datetime.now()
DATE_TODAY_STR = TODAY.strftime("%Y%m%d")
X_DAYS_AGO = TODAY - timedelta(days=UPDATE_INTERVAL)
X_DAYS_AGO_STR = X_DAYS_AGO.strftime("%Y%m%d")

Get CSW records

In [27]:
import requests
import xml.etree.ElementTree as ET
import json

def fetch_records(base_url, max_records=100):
    records = []
    start_position = 1
    
    while True:
        # Construct the request URL
        url = f"{base_url}&maxRecords={max_records}&startPosition={start_position}"
        print(f"Fetching: {url}")
        
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Error: {response.status_code}")
            break
        
        # Parse XML response
        root = ET.fromstring(response.content)
        
        # Find all records in the response
        ns = {'csw': 'http://www.opengis.net/cat/csw/2.0.2', 'dc': 'http://purl.org/dc/elements/1.1/', 'dct': 'http://purl.org/dc/terms/'}
        record_elements = root.findall('.//csw:Record', ns)
        
        if not record_elements:
            print("No more records found. Stopping.")
            break
        
        for record in record_elements:
            record_data = {}
            
            # Extract single-value elements
            for tag in ['identifier', 'date', 'title', 'type', 'format', 'description']:
                element = record.find(f'dc:{tag}', ns)
                if element is not None:
                    record_data[tag] = element.text.strip() if element.text else None
            
            for tag in ['modified', 'abstract']:
                element = record.find(f'dct:{tag}', ns)
                if element is not None:
                    record_data[tag] = element.text.strip() if element.text else None
            

            # Extract multi-value elements as lists
            for tag in ['subject', 'rights']:
                elements = record.findall(f'dc:{tag}', ns)
                record_data[tag] = [e.text.strip() for e in elements if e.text]
            
            # Extract URIs with descriptions
            uris = []
            for uri in record.findall('dc:URI', ns):
                uri_data = {"url": uri.text.strip() if uri.text else None}
                if 'description' in uri.attrib:
                    uri_data["description"] = uri.attrib['description']
                uris.append(uri_data)
            record_data['URI'] = uris
            
            records.append(record_data)
        
        start_position += max_records
    
    return records


CSW_URL = "https://gdi.berlin.de/geonetwork/srv/ger/csw?service=CSW&version=2.0.2&request=GetRecords&resultType=results&elementSetName=full&typeNames=csw:Record"
all_records = fetch_records(CSW_URL, max_records=100)

# Save to JSON file
with open(FILE_NAME_CSW, "w", encoding="utf-8") as f:
    json.dump(all_records, f, ensure_ascii=False, indent=4)

print(f"Saved {len(all_records)} records to csw_records.json")


Fetching: https://gdi.berlin.de/geonetwork/srv/ger/csw?service=CSW&version=2.0.2&request=GetRecords&resultType=results&elementSetName=full&typeNames=csw:Record&maxRecords=100&startPosition=1


KeyboardInterrupt: 

### CSW Dataset Sync & Logging

This script processes a list of CSW records (from a JSON file), manages corresponding dataset folders, and logs any updates or deletions:

    ✅ Creates or updates dataset folders in ../datasets/ if:

        The dataset is new (no folder exists), or

        The modified date is within the last 10 days

    📁 Saves each dataset as record.json in its corresponding folder

    🆕 Logs updates (written datasets) to ../data/logs/updates/updates_YYYYMMDD.json

    🗑️ Detects deletions: if a folder exists for a dataset no longer present in the CSW file, its ID is logged to ../data/logs/deletions/deletions_YYYYMMDD.json

    🛑 Skips records without a guid or invalid/missing modified field

This ensures your local dataset collection is synced with the latest CSW records and changes are tracked for auditing or cleanup.

In [28]:
import json
import os

# === Load CSW JSON ===
with open(FILE_NAME_CSW, 'r', encoding='utf-8') as file:
    data = json.load(file)

# === For logging ===
updated_ids = []
csw_ids = set()

# === Process records ===
for obj in data:
    identifier = obj.get('identifier', None)
    if not identifier:
        print(f"⚠️ Skipping object: no 'guid' found.")
        continue  # Skip this object entirely

    type = obj.get("type", "")
    if type != "service":
        print(f"⚠️ Skipping CSW entry which is not a service but a {type}")
        continue  # Skip this object entirely
    
    csw_ids.add(identifier)  # Track what should exist

    date_modified = obj.get('modified', None)
    should_write = False

    folder_path = os.path.join(BASE_DIR_DATASETS, identifier)
    output_file = os.path.join(folder_path, "csw_record.json")

    if not os.path.exists(folder_path):
        # === NEW DATASET ===
        should_write = True
        print(f"📁 New dataset: {identifier} (no folder yet). Will create and save.")
    else:
        if date_modified:
            # === EXISTING: only update if recent ===
            try:
                incoming_date = datetime.fromisoformat(date_modified)
                if incoming_date >= X_DAYS_AGO:
                    should_write = True
                    print(f"🆕 Updated: {identifier} is recent (modified {date_modified} >= {X_DAYS_AGO.date()}) → Will overwrite.")
                else:
                    print(f"⏭️ Older than 10 days: {identifier} (modified {date_modified} < {X_DAYS_AGO.date()}) → No update.")
            except ValueError:
                print(f"⚠️ Invalid date format for {identifier}: {date_modified} → Skipping update.")
        else:
            # === EXISTING & NO MODIFIED ===
            print(f"❌ No modified date for {identifier} → Skipping update. (also triggered when duplicate)")

    if should_write:
        os.makedirs(folder_path, exist_ok=True)
        with open(output_file, 'w', encoding='utf-8') as out_file:
            json.dump(obj, out_file, indent=2, ensure_ascii=False)
        updated_ids.append(identifier)
        print(f"✅ Saved {identifier} to {output_file}")

# === Find deleted datasets ===
existing_folders = set(os.listdir(BASE_DIR_DATASETS))
deleted_ids = list(existing_folders - csw_ids)

# === Make sure log folders exist ===
os.makedirs(UPDATES_LOG_DIR, exist_ok=True)
os.makedirs(DELETIONS_LOG_DIR, exist_ok=True)


updates_log_file = os.path.join(UPDATES_LOG_DIR, f"updates_{DATE_TODAY_STR}.json")
deletions_log_file = os.path.join(DELETIONS_LOG_DIR, f"deletions_{DATE_TODAY_STR}.json")

with open(updates_log_file, 'w', encoding='utf-8') as f:
    json.dump(updated_ids, f, indent=2, ensure_ascii=False)
print(f"📝 Updates log written to {updates_log_file}")

with open(deletions_log_file, 'w', encoding='utf-8') as f:
    json.dump(deleted_ids, f, indent=2, ensure_ascii=False)
print(f"🗑️ Deletions log written to {deletions_log_file}")


📁 New dataset: 633bafd0-f522-34bd-af4e-32a2cf72fb0c (no folder yet). Will create and save.
✅ Saved 633bafd0-f522-34bd-af4e-32a2cf72fb0c to ./data/datasets/633bafd0-f522-34bd-af4e-32a2cf72fb0c/csw_record.json
📁 New dataset: 955827a2-147d-3591-b376-54e633803817 (no folder yet). Will create and save.
✅ Saved 955827a2-147d-3591-b376-54e633803817 to ./data/datasets/955827a2-147d-3591-b376-54e633803817/csw_record.json
📁 New dataset: 6ef7e20a-1e39-3c95-9ef7-0b57bb99f55a (no folder yet). Will create and save.
✅ Saved 6ef7e20a-1e39-3c95-9ef7-0b57bb99f55a to ./data/datasets/6ef7e20a-1e39-3c95-9ef7-0b57bb99f55a/csw_record.json
📁 New dataset: 46cb329c-6b1c-35d5-a87c-eab60b07f301 (no folder yet). Will create and save.
✅ Saved 46cb329c-6b1c-35d5-a87c-eab60b07f301 to ./data/datasets/46cb329c-6b1c-35d5-a87c-eab60b07f301/csw_record.json
📁 New dataset: 2d145060-2af2-38ec-8281-08eb2c3eb5da (no folder yet). Will create and save.
✅ Saved 2d145060-2af2-38ec-8281-08eb2c3eb5da to ./data/datasets/2d145060-2af2

### Read in the ids to update and delete  

In [29]:
import json
import os
from datetime import datetime

# Log file to read
updates_log_file = os.path.join(UPDATES_LOG_DIR, f"updates_{DATE_TODAY_STR}.json")
deletions_log_file = os.path.join(DELETIONS_LOG_DIR, f"deletions_{DATE_TODAY_STR}.json")

# === Read the log files ===
with open(updates_log_file, 'r') as f:
    UPDATES_IDS = json.load(f)

with open(deletions_log_file, 'r') as f:
    DELETION_IDS = json.load(f)

print(f"✅ Loaded log with {len(UPDATES_IDS)} UPDATES_IDS from {updates_log_file}")
print(f"✅ Loaded log with {len(DELETION_IDS)} DELETION_IDS from {deletions_log_file}")


✅ Loaded log with 2070 UPDATES_IDS from ./data/logs/updates/updates_20250626.json
✅ Loaded log with 0 DELETION_IDS from ./data/logs/deletions/deletions_20250626.json


Scrape all the Data

In [30]:
import os
import json
from owslib.wfs import WebFeatureService
import importlib

import helpers
importlib.reload(helpers)
from helpers import get_html_as_markdown, get_wfs_layer_attributes_simple, strip_query,clean_keywords, get_attribute_descriptions_from_describe_featuretype


# Load the attributesDescription.json
# with open("../data/attributesDescription.json", "r") as f:
#     attributes_descriptions = json.load(f)

# === Updated loading loop ===
for identifier in updated_ids:
    
    folder_path = os.path.join(BASE_DIR_DATASETS, identifier)
    record_file = os.path.join(folder_path, "csw_record.json")

    # Initialize with safe defaults
    service_data = {
        "title": "",
        "description": "",
        "abstract": "",
        "guid": "",
        "keywords": [],
        "serviceType": "Unknown",
        "serviceUrl": None
    }

    if os.path.exists(record_file):
        with open(record_file, 'r') as f:
            record_data = json.load(f)

            # Fill main metadata
            service_data["title"] = record_data.get("title", "")
            service_data["description"] = record_data.get("description", "")
            service_data["abstract"] = record_data.get("abstract", "")
            service_data["guid"] = record_data.get("identifier", "")

            # Filter tags
            subject = record_data.get("subject", [])
            service_data["keywords"] = clean_keywords(subject)

            # # Load attributes description if present
            # attr_desc = attributes_descriptions.get(service_data["guid"])
            # if attr_desc:
            #     service_data["attributesDescription"] = attr_desc

            # Detect service URLs and markdown text
            wfs = None
            wms = None
            description_url = None
            description_text = None

            uris = record_data.get("URI", [])
            for uri in uris:
                url = uri.get("url", "")
                if "service=wfs" in url.lower():
                    wfs = strip_query(url)
                elif "service=wms" in url.lower():
                    wms = url
                
                description = uri.get("description", "")
                if description == "Technische Beschreibung":
                    # Ignore if it ends with .pdf (case-insensitive)
                    if not url.endswith(".pdf"):
                        description_url = url
                        description_text = get_html_as_markdown(description_url,identifier)
                        if description_text:
                            service_data["markdownText"] = description_text

            if description_url:
                service_data["description_url"] = description_url

            # ✅ Set service type and URL immediately
            if wfs:
                service_data["serviceType"] = "WFS"
                service_data["serviceUrl"] = wfs
            elif wms:
                service_data["serviceType"] = "WMS"
                service_data["serviceUrl"] = wms

            # === WFS-specific metadata ===
            if wfs:
                try:
                    wfs_result = WebFeatureService(url=wfs)
                    layers = list(wfs_result.contents)

                    if len(layers) > 1:
                        service_data["layers"] = []
                        for layer_name in layers:

                            feature_type = wfs_result.contents[layer_name]
                            cleaned_keywords = clean_keywords(feature_type.keywords)
                            layer = {
                                "title": feature_type.title,
                                "abstract": feature_type.abstract,
                                "keywords": cleaned_keywords,
                                "typeName": layer_name,
                            }

                            # ✅ Try get_schema separately: never stop the loop
                            try:
                                schema = wfs_result.get_schema(layer_name)
                                layer["attributes"] = list(schema['properties'].keys())
                            except Exception as e:
                                print(f"⚠️ Can not access layer schema {identifier} - {layer_name}: {e}")
                                layer["attributes"] = []

                            layer["attributesDescriptions"] = get_attribute_descriptions_from_describe_featuretype(wfs,layer_name)

                            service_data["layers"].append(layer)


                    elif layers:
                        # Single layer
                        layer_name = layers[0]
                        service_data["attributes"] = get_wfs_layer_attributes_simple(wfs, layer_name)
                        service_data["typeName"] = layer_name
                        service_data["attributesDescriptions"] = get_attribute_descriptions_from_describe_featuretype(wfs,layer_name)

                except Exception as e:
                    print(f"⚠️ Failed to read WFS for {identifier}: {e}")

    else:
        print(f"⚠️ Missing CSW entry for {identifier} in {folder_path}")
        # return / ignore?

    # === Write final JSON ===
    output_file = os.path.join(folder_path, "collected_data.json")
    with open(output_file, 'w', encoding='utf-8') as out_file:
        json.dump(service_data, out_file, indent=2, ensure_ascii=False)
        sType = service_data["serviceType"]
        print(f"✅ Written {sType} with id {identifier}")


✅ Written WMS with id 633bafd0-f522-34bd-af4e-32a2cf72fb0c
✅ Written WMS with id 955827a2-147d-3591-b376-54e633803817
✅ Written WMS with id 6ef7e20a-1e39-3c95-9ef7-0b57bb99f55a
✅ Written WFS with id 46cb329c-6b1c-35d5-a87c-eab60b07f301
✅ Written WFS with id 2d145060-2af2-38ec-8281-08eb2c3eb5da
✅ Written WMS with id 5ff506d1-38a7-3f24-9e31-f1ef731c4e5a
✅ Written WFS with id eeac62c9-0412-3b35-9b0a-0d0eaef6ad12
✅ Written WMS with id 4af2671f-304e-345e-a988-ee452f08864b
✅ Written WMS with id 311d64ce-ac40-3843-9c96-c5f652d51122
✅ Written WMS with id 9f025795-edeb-3205-b7fa-3a60925dbea9
✅ Written WFS with id 3b580b1c-f1cb-350f-bc13-0fccffb594b9
✅ Written WMS with id a7438b11-7e4e-3dc8-a434-d4e3a99a0b9a
✅ Written WMS with id 1cd8baab-4378-3b54-9003-740f11888a1e
✅ Written WMS with id 38305570-e0fe-3e19-a0fa-ad503c8e0136
✅ Written WMS with id 47037e82-033a-307e-8ea1-192c5b9b183d
✅ Written WMS with id f02bc957-0266-356e-bdff-ca2e7488de38
✅ Written WFS with id aedfff1e-4f09-3cfc-938f-a8f90f4bef

### Write Markdown and JSON
Only WMS and WFS

In [ ]:
import os
import json


os.makedirs(BASE_DIR_MARKDOWNS, exist_ok=True)

def writeMarkdown(record_data, layer=None, identifier=None):

    markdown_lines = []

    json_for_db = {}
    json_for_db["Typ"] = record_data.get("serviceType")
    json_for_db["guid"] = record_data.get("guid")
    json_for_db["Service URL"] = record_data.get("serviceUrl")

    # Fix: get typeName from layer if available
    if layer and layer.get("typeName"):
        json_for_db["Layer Name"] = layer["typeName"]
    elif record_data.get("typeName"):
        json_for_db["Layer Name"] = record_data["typeName"]


    # === 1️⃣ Use layer's title if present, else record_data's ===
    title = (layer.get("title") if layer else None) or record_data.get("title", "")
    if title:
        title = title.replace('- [WFS]', '').replace(' (Umweltatlas)', '').replace('- [WMS]', '').replace(' (Linien)', '')
        markdown_lines.append(f"# {title}\n")

        json_for_db["Titel"] = title

    # === 2️⃣ Only add description if there is NO layer ===
    if record_data.get("description"):
        markdown_lines.append("## Beschreibung\n")
        markdown_lines.append(record_data["description"])
        markdown_lines.append("")

        json_for_db["Anmerkung"] = record_data["description"]


    # === 3️⃣ Use layer's abstract if present, else record_data's ===
    abstract = (layer.get("abstract") if layer else None)
    if abstract:
        markdown_lines.append("## Layer Beschreibung\n")
        markdown_lines.append(abstract)
        markdown_lines.append("")

        json_for_db["Anmerkung Layer"] = abstract


    # === 4️⃣ Use ONLY layer's attributesDescriptions if layer exists ===
    attributesDescriptions = layer.get("attributesDescriptions") if layer else None
    if attributesDescriptions:
        markdown_lines.append("## Attribut-Tabelle\n")
        markdown_lines.append("| Feldname | Beschreibung |")
        markdown_lines.append("| --- | --- |")
        for feldname, beschreibung in attributesDescriptions.items():
            markdown_lines.append(f"| {feldname} | {beschreibung} |")
        markdown_lines.append("")

        json_for_db["Attribute"] = attributesDescriptions

    else:
        # If no layer or no layer attributesDescriptions,
        # fall back to record_data attributes (NOT attributesDescriptions!)
        attributes = layer.get("attributes") if layer else record_data.get("attributes", [])
        if attributes:
            markdown_lines.append("## Attribute\n")
            markdown_lines.append(", ".join(k.strip() for k in attributes))
            markdown_lines.append("")

            json_for_db["Attribute"] = attributes

    # === 5️⃣ Use ONLY layer's keywords if present ===
    keywords = layer.get("keywords") if layer else record_data.get("keywords", [])
    if keywords:
        markdown_lines.append("## Schlagwörter\n")
        markdown_lines.append(", ".join(k.strip() for k in keywords))
        markdown_lines.append("")

        json_for_db["Stichworte"] = keywords



    # === 6️⃣ markdownText only comes from record_data ===
    markdownText = record_data.get("markdownText", "")
    if markdownText:
        cutoff_phrase = "Ausführliche Informationen finden Sie in den Begleittexten"
        markdownText = markdownText.split(cutoff_phrase)[0].strip()
        markdown_lines.append("\n## Technische Beschreibung\n")
        markdown_lines.append(markdownText)
        markdown_lines.append("")

        json_for_db["Beschreibung"] = markdownText


    # === 7️⃣ Join and write ===
    markdown_text = "\n".join(markdown_lines)
    typeName = ("_" + layer.get("typeName")) if layer else ""

    file_path_markdown = os.path.join(BASE_DIR_MARKDOWNS, f"{identifier}{typeName}.md")
    with open(file_path_markdown, "w", encoding="utf-8") as f:
        f.write(markdown_text)

    file_path_json = os.path.join(BASE_DIR_MARKDOWNS, f"{identifier}{typeName}.json")
    with open(file_path_json, "w", encoding="utf-8") as f:
        json.dump(json_for_db, f, indent=2, ensure_ascii=False)
                
    print("✅ Markdown file saved:", f"{identifier}{typeName}")


# === Loop through UPDATES_IDS and layers ===
for identifier in UPDATES_IDS:
    folder_path = os.path.join(BASE_DIR_DATASETS, identifier)
    record_file = os.path.join(folder_path, "collected_data.json")

    if os.path.exists(record_file):
        with open(record_file, 'r', encoding="utf-8") as f:
            record_data = json.load(f)

        # Only WFS and WMS
        if record_data.get("serviceType") in ["WMS", "WFS"]:
            layers = record_data.get("layers")
            if layers:
                for layer in layers:
                    writeMarkdown(record_data, layer, identifier)
            else:
                writeMarkdown(record_data, None, identifier)


✅ Markdown file saved: 633bafd0-f522-34bd-af4e-32a2cf72fb0c
✅ Markdown file saved: 955827a2-147d-3591-b376-54e633803817
✅ Markdown file saved: 6ef7e20a-1e39-3c95-9ef7-0b57bb99f55a
✅ Markdown file saved: 46cb329c-6b1c-35d5-a87c-eab60b07f301
✅ Markdown file saved: 2d145060-2af2-38ec-8281-08eb2c3eb5da
✅ Markdown file saved: 5ff506d1-38a7-3f24-9e31-f1ef731c4e5a
✅ Markdown file saved: eeac62c9-0412-3b35-9b0a-0d0eaef6ad12
✅ Markdown file saved: 4af2671f-304e-345e-a988-ee452f08864b
✅ Markdown file saved: 311d64ce-ac40-3843-9c96-c5f652d51122
✅ Markdown file saved: 9f025795-edeb-3205-b7fa-3a60925dbea9
✅ Markdown file saved: 3b580b1c-f1cb-350f-bc13-0fccffb594b9
✅ Markdown file saved: a7438b11-7e4e-3dc8-a434-d4e3a99a0b9a
✅ Markdown file saved: 1cd8baab-4378-3b54-9003-740f11888a1e
✅ Markdown file saved: 38305570-e0fe-3e19-a0fa-ad503c8e0136
✅ Markdown file saved: 47037e82-033a-307e-8ea1-192c5b9b183d
✅ Markdown file saved: f02bc957-0266-356e-bdff-ca2e7488de38
✅ Markdown file saved: aedfff1e-4f09-3cf

## Create Embeddings

In [ ]:
import os
import uuid
import glob
from urllib.parse import quote
from openai import OpenAI
from supabase import create_client, Client
from dotenv import load_dotenv

# Load env vars
load_dotenv(override=True)

# Initialize clients
client = OpenAI(api_key=os.environ["OPENAI_KEY"])
supabase: Client = create_client(
    os.environ["SUPABASE_URL"],
    os.environ["SUPABASE_SERVICE_ROLE_KEY"]
)

# 1️⃣ Read Markdown files
markdown_files = glob.glob("./data/markdowns/*.md", recursive=True)
# markdown_files = markdown_files[:10]
print(markdown_files)

for filepath in markdown_files:
    with open(filepath, "r", encoding="utf-8") as f:
        content = f.read()

    filepath_json = filepath.replace(".md", ".json")

    with open(filepath_json, "r", encoding="utf-8") as f:
        content_json = f.read()

    # Create embedding
    response = client.embeddings.create(
        model="text-embedding-3-small",
        input=content
    )
    embedding = response.data[0].embedding  # List[float]

    # Generate ID & metadata
    id = str(uuid.uuid4())
    slug = os.path.basename(filepath).replace(".md", "")
    slug = quote(slug, safe="")
    heading = content.split("\n")[0].strip()
    dataset_info = content_json

    # Insert to Supabase
    data = {
        "id": id,
        "slug": slug,
        "heading": heading,
        "dataset_info": dataset_info,
        "embedding": embedding
    }

    res = supabase.table("nods_page_section_v2").insert(data).execute()
    print(f"Inserted: {filepath} → {res}")

print("✅ All files embedded and stored!")


## For the Graph 

### Download data

In [40]:
import pandas as pd  # Make sure pandas is installed
from supabase import create_client, Client
from dotenv import load_dotenv

# Load env vars
load_dotenv(override=True)

supabase: Client = create_client(
    os.environ["SUPABASE_URL"],
    os.environ["SUPABASE_SERVICE_ROLE_KEY"]
)

# Fetch data from Supabase table
res = supabase.table("nods_page_section_v2").select("*").execute()

data = res.data  # This is a list of dictionaries (rows)

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv("./data/nods_page_section_v2.csv", index=False)
print("✅ Data saved to nods_page_section_v2.csv")


✅ Data saved to nods_page_section_v2.csv


### Create a t-SNE model

In [42]:
import pandas as pd
from sklearn.manifold import TSNE
import numpy as np
from ast import literal_eval

# Load the embeddings
datafile_path = "./data/nods_page_section_v2.csv"
df = pd.read_csv(datafile_path)

# Convert to a list of lists of floats
matrix = np.array(df.embedding.apply(literal_eval).to_list())

# Create a t-SNE model and transform the data
tsne = TSNE(n_components=2, perplexity=15, random_state=42, init='random', learning_rate=200)
vis_dims = tsne.fit_transform(matrix)
vis_dims.shape

(1000, 2)

### Visualize (not needed)

In [44]:
# import matplotlib.pyplot as plt

# # Plot the data
# plt.figure(figsize=(8, 6))
# plt.scatter(vis_dims[:, 0], vis_dims[:, 1], alpha=0.5)
# plt.title('t-SNE Visualization')
# plt.xlabel('Dimension 1')
# plt.ylabel('Dimension 2')
# plt.grid(True)
# plt.show()

### Create CSV for GeoExplorer

In [ ]:
import pandas as pd
from IPython.display import FileLink  # Import FileLink from IPython.display

# Load the embeddings
datafile_path = "nods_page_section_v2_rows.csv"
df = pd.read_csv(datafile_path)

titles = df['heading'].str.split('#').str[1].str.split('*').str[0].str.strip().str.replace('`','')

# Create a DataFrame with the t-SNE transformed data and content column
tsne_df = pd.DataFrame(vis_dims, columns=['Dimension 1', 'Dimension 2'])
tsne_df['Slug'] = df['slug']  # Assuming the column name in the original data is 'content'
tsne_df['Title'] = titles

# Export the DataFrame to a CSV file
tsne_df.to_csv('tsne_data.csv', index=False)

# Create a link for downloading the CSV file
FileLink('./data/tsne_data.csv')

### Create lots of small CSVs with 1000 entries each for upload

In [ ]:
# import pandas as pd
# import os

# # File name of the source CSV
# input_file = './nods_page_section_v2_rows.csv'
# chunk_size = 1000
# output_prefix = 'nods_part_'

# df = pd.read_csv(input_file)

# # Determine how many chunks are needed
# total_rows = len(df)
# num_chunks = (total_rows + chunk_size - 1) // chunk_size  # Ceiling division

# # Create output files
# for i in range(num_chunks):
#     start_row = i * chunk_size
#     end_row = start_row + chunk_size
#     chunk_df = df[start_row:end_row]

#     output_file = f"{output_prefix}{i + 1}.csv"
#     chunk_df.to_csv(output_file, index=False)
#     print(f"Created: {output_file} with rows {start_row} to {min(end_row - 1, total_rows - 1)}")


